notebook that takes harmonic and noise part of the resynthesis and takes the relevant part of each. 

#getting the signals 


In [ ]:
#mount the drive
from google.colab import drive
drive.mount('/content/drive')
HOME="/content/drive/My Drive"

In [ ]:
%cd /content
!rm -rf ddsp_gm2  
!git clone https://gianmarcohutter/ddsp_gm2/
%cd ddsp_gm2

In [ ]:
%tensorflow_version 2.x
print('Installing from pip package...')
#!pip install -qU ddsp
!pip install -e /content/ddsp_gm2[ddsp] 
!pip install mir_eval
# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import time

import crepe
import ddsp
import ddsp.training
from ddsp.colab import colab_utils
from ddsp.colab.colab_utils import (
    auto_tune, detect_notes, fit_quantile_transform, 
    get_tuning_factor, download, play, record, 
    specplot, upload, DEFAULT_SAMPLE_RATE)
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pickle
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Helper Functions
sample_rate = DEFAULT_SAMPLE_RATE  # 16000


from google.colab import output
def beep():
  output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')


print('Done!')

In [ ]:
#@title Record or Upload Audio
#@markdown * Either record audio from microphone or upload audio from file (.mp3 or .wav) 
#@markdown * Audio should be monophonic (single instrument / voice)
#@markdown * Extracts fundmanetal frequency (f0) and loudness features. 
#beep()
record_or_upload = "Upload (.mp3 or .wav)"  #@param ["Record", "Upload (.mp3 or .wav)"]

record_seconds =     5#@param {type:"number", min:1, max:10, step:1}

if record_or_upload == "Record":
  audio = record(seconds=record_seconds)
else:
  # Load audio sample here (.mp3 or .wav3 file)
  # Just use the first file.
  filenames, audios = upload()
  audio = audios[0]
audio = audio[np.newaxis, :]
print('\nExtracting audio features...')

# Plot.
specplot(audio)
play(audio)

# Setup the session.
ddsp.spectral_ops.reset_crepe()

# Compute features.
start_time = time.time()
audio_features = ddsp.training.metrics.compute_audio_features(audio)
audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
audio_features_mod = None
print('Audio features took %.1f seconds' % (time.time() - start_time))


TRIM = -15
# Plot Features.
fig, ax = plt.subplots(nrows=3, 
                       ncols=1, 
                       sharex=True,
                       figsize=(6, 8))
ax[0].plot(audio_features['loudness_db'][:TRIM])
ax[0].set_ylabel('loudness_db')

ax[1].plot(librosa.hz_to_midi(audio_features['f0_hz'][:TRIM]))
ax[1].set_ylabel('f0 [midi]')

ax[2].plot(audio_features['f0_confidence'][:TRIM])
ax[2].set_ylabel('f0 confidence')
_ = ax[2].set_xlabel('Time step [frame]')

In [ ]:
#going back from ddsp_gm2
%cd /content

In [ ]:

#put the path to the drive folder here: 
#fnames="drive/My\ Drive/test5/ddsp-solo-instrument/my_solo_instrument.zip"


#these three are needed to control what model to take
test_folder="drive/My\ Drive/test49/GM-Voice/"
try_intermediate_training = True #choose true if you want to listen to another model than whats in the zip
training_steps=32400

fnames=test_folder+"GM-Voice.zip"
intermediate_checkpoint_index=test_folder+"ckpt-"+str(training_steps)+".index"
intermediate_checkpoint_data=test_folder+"ckpt-"+str(training_steps)+".data-00000-of-00001"
statistics=test_folder+"dataset_statistics.pkl"
config=test_folder+"operative_config*"

def find_model_dir(dir_name):
  # Iterate through directories until model directory is found
  for root, dirs, filenames in os.walk(dir_name):
    for filename in filenames:
      if filename.endswith(".gin") and not filename.startswith("."):
        model_dir = root
        break
  return model_dir

def find_model_file(dir_name):
  # Iterate through directories until model directory is found
  for root, dirs, filenames in os.walk(dir_name):
    for filename in filenames:
      if filename.endswith(".gin") and not filename.startswith("."):
        model_file = filename
        break
  return model_file  


# User models.
UPLOAD_DIR = '/content/uploaded'
!rm -r $UPLOAD_DIR
!mkdir $UPLOAD_DIR


if try_intermediate_training == False:
  #just take whats in the .zip
  print("Unzipping... {}".format(fnames))
  !unzip -o $fnames -d $UPLOAD_DIR &> /dev/null
else:
  #copy the required files to the upload folder
  !cp $intermediate_checkpoint_index $UPLOAD_DIR
  !cp $intermediate_checkpoint_data $UPLOAD_DIR
  !cp $statistics $UPLOAD_DIR
  !cp $config $UPLOAD_DIR

model_dir = find_model_dir(UPLOAD_DIR)
model_file= find_model_file(UPLOAD_DIR)
#gin_file = os.path.join(model_dir, 'operative_config-0.gin')
gin_file = os.path.join(model_dir, model_file)

# Load the dataset statistics.
DATASET_STATS = None
dataset_stats_file = os.path.join(model_dir, 'dataset_statistics.pkl')
print(f'Loading dataset statistics from {dataset_stats_file}')
try:
  if tf.io.gfile.exists(dataset_stats_file):
    with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
      DATASET_STATS = pickle.load(f)
except Exception as err:
  print('Loading dataset statistics from pickle failed: {}.'.format(err))


# Parse gin config,
with gin.unlock_config():
  gin.parse_config_file(gin_file, skip_unknown=True)  

# Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
ckpt_name = ckpt_files[0].split('.')[0]
ckpt = os.path.join(model_dir, ckpt_name)

# Ensure dimensions and sampling rates are equal
time_steps_train = gin.query_parameter('DefaultPreprocessor.time_steps')
n_samples_train = gin.query_parameter('Additive.n_samples')
hop_size = int(n_samples_train / time_steps_train)

time_steps = int(audio.shape[1] / hop_size)
n_samples = time_steps * hop_size

print("===Trained model===")
print("Time Steps", time_steps_train)
print("Samples", n_samples_train)
print("Hop Size", hop_size)
print("\n===Resynthesis===")
print("Time Steps", time_steps)
print("Samples", n_samples)
print('')


# Trim all input vectors to correct lengths 
for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
  audio_features[key] = audio_features[key][:time_steps]
audio_features['audio'] = audio_features['audio'][:, :n_samples]


gin_params = [
    'RnnFcDecoder.input_keys = ("ld_scaled", "f0_scaled", "z")',
    #'RnnFcDecoder.input_keys = ( "ld_scaled","z")',
    'Additive.n_samples = {}'.format(n_samples),
    'FilteredNoise.n_samples = {}'.format(n_samples),
    'DefaultPreprocessor.time_steps = {}'.format(time_steps),
    "ProcessorGroup.dag = \
    [(@synths.Additive(), ['amps', 'harmonic_distribution', 'f0_hz'])]"
]

with gin.unlock_config():
  gin.parse_config(gin_params)

#GM create two separate models
# Set up the model just to predict audio given new conditioning
model1 = ddsp.training.models.Autoencoder()
model1.restore(ckpt)

# Build model by running a batch through it.
_ = model1(audio_features, training=False)

gin_params2 = [
    'RnnFcDecoder.input_keys = ("ld_scaled", "f0_scaled", "z")',
    #'RnnFcDecoder.input_keys = ("ld_scaled","z")',
    'Additive.n_samples = {}'.format(n_samples),
    'FilteredNoise.n_samples = {}'.format(n_samples),
    'DefaultPreprocessor.time_steps = {}'.format(time_steps),
    "ProcessorGroup.dag = \
    [(@synths.FilteredNoise(), ['noise_magnitudes'])]"
]

with gin.unlock_config():
  gin.parse_config(gin_params2)

model2 = ddsp.training.models.Autoencoder()
model2.restore(ckpt)
_ = model2(audio_features, training=False)

In [ ]:
#@title Modify conditioning

#@markdown These models were not explicitly trained to perform timbre transfer, so they may sound unnatural if the incoming loudness and frequencies are very different then the training data (which will always be somewhat true). 


#@markdown ## Note Detection

#@markdown You can leave this at 1.0 for most cases
threshold = 1 #@param {type:"slider", min: 0.0, max:2.0, step:0.01}


#@markdown ## Automatic

ADJUST = True #@param{type:"boolean"}

#@markdown Quiet parts without notes detected (dB)
quiet = 20 #@param {type:"slider", min: 0, max:60, step:1}

#@markdown Force pitch to nearest note (amount)
autotune = 0 #@param {type:"slider", min: 0.0, max:1.0, step:0.1}

#@markdown ## Manual


#@markdown Shift the pitch (octaves)
pitch_shift =  -1 #@param {type:"slider", min:-2, max:2, step:0.1}

#@markdown Adjsut the overall loudness (dB)
loudness_shift = 20 #@param {type:"slider", min:-20, max:20, step:1}

for k, v in audio_features.items():
  print(k)
  print(v)
audio_features_mod = {k: v.copy() for k, v in audio_features.items()}


## Helper functions.
def shift_ld(audio_features, ld_shift=0.0):
  """Shift loudness by a number of ocatves."""
  audio_features['loudness_db'] += ld_shift
  return audio_features


def shift_f0(audio_features, pitch_shift=0.0):
  print(pitch_shift)
  """Shift f0 by a number of ocatves."""
  audio_features['f0_hz'] *= 2.0 ** (pitch_shift)
  audio_features['f0_hz'] = np.clip(audio_features['f0_hz'], 
                                    0.0, 
                                    librosa.midi_to_hz(110.0))
  return audio_features


mask_on = None

if ADJUST and DATASET_STATS is not None:
  # Detect sections that are "on".
  mask_on, note_on_value = detect_notes(audio_features['loudness_db'],
                                        audio_features['f0_confidence'],
                                        threshold)
  
  if np.any(mask_on):
    # Shift the pitch register.
    target_mean_pitch = DATASET_STATS['mean_pitch']
    pitch = ddsp.core.hz_to_midi(audio_features['f0_hz'])
    mean_pitch = np.mean(pitch[mask_on])
    p_diff = target_mean_pitch - mean_pitch
    p_diff_octave = p_diff / 12.0
    round_fn = np.floor if p_diff_octave > 1.5 else np.ceil
    p_diff_octave = round_fn(p_diff_octave)
    audio_features_mod = shift_f0(audio_features_mod, p_diff_octave)


    # Quantile shift the note_on parts.
    _, loudness_norm = colab_utils.fit_quantile_transform(
        audio_features['loudness_db'],
        mask_on,
        inv_quantile=DATASET_STATS['quantile_transform'])

    # Turn down the note_off parts.
    mask_off = np.logical_not(mask_on)
    loudness_norm[mask_off] -=  quiet * (1.0 - note_on_value[mask_off][:, np.newaxis])
    loudness_norm = np.reshape(loudness_norm, audio_features['loudness_db'].shape)
    
    audio_features_mod['loudness_db'] = loudness_norm 

    # Auto-tune.
    if autotune:
      f0_midi = np.array(ddsp.core.hz_to_midi(audio_features_mod['f0_hz']))
      tuning_factor = get_tuning_factor(f0_midi, audio_features_mod['f0_confidence'], mask_on)
      f0_midi_at = auto_tune(f0_midi, tuning_factor, mask_on, amount=autotune)
      audio_features_mod['f0_hz'] = ddsp.core.midi_to_hz(f0_midi_at)

  else:
    print('\nSkipping auto-adjust (no notes detected or ADJUST box empty).')

else:
  print('\nSkipping auto-adujst (box not checked or no dataset statistics found).')

# Manual Shifts.
audio_features_mod = shift_ld(audio_features_mod, loudness_shift)
audio_features_mod = shift_f0(audio_features_mod, pitch_shift)



# Plot Features.
has_mask = int(mask_on is not None)
n_plots = 3 if has_mask else 2 
fig, axes = plt.subplots(nrows=n_plots, 
                      ncols=1, 
                      sharex=True,
                      figsize=(2*n_plots, 8))

if has_mask:
  ax = axes[0]
  ax.plot(np.ones_like(mask_on[:TRIM]) * threshold, 'k:')
  ax.plot(note_on_value[:TRIM])
  ax.plot(mask_on[:TRIM])
  ax.set_ylabel('Note-on Mask')
  ax.set_xlabel('Time step [frame]')
  ax.legend(['Threshold', 'Likelihood','Mask'])

ax = axes[0 + has_mask]
ax.plot(audio_features['loudness_db'][:TRIM])
ax.plot(audio_features_mod['loudness_db'][:TRIM])
ax.set_ylabel('loudness_db')
ax.legend(['Original','Adjusted'])

ax = axes[1 + has_mask]
ax.plot(librosa.hz_to_midi(audio_features['f0_hz'][:TRIM]))
ax.plot(librosa.hz_to_midi(audio_features_mod['f0_hz'][:TRIM]))
ax.set_ylabel('f0 [midi]')
_ = ax.legend(['Original','Adjusted'])

print("type:" + str(type(audio_features)))
print(audio_features.keys())

In [ ]:
#@title #Resynthesize Audio


af = audio_features if audio_features_mod is None else audio_features_mod

# Run a batch of predictions.
start_time = time.time()

harmonic_tensor = model1(af, training=False)
noise_tensor = model2(af, training=False)
print('Prediction took %.1f seconds' % (time.time() - start_time))

# Plot
print('Original')
play(audio)

print('Harmonic Resynthesis')
play(harmonic_tensor)

print('Noise Resynthesis')
play(noise_tensor)

specplot(audio)
plt.title("Original")

specplot(harmonic_tensor)
_ = plt.title("Harmonic")

specplot(noise_tensor)
_ = plt.title("Noise")

#mixing the two signals
parameters to change:
* CROSSFADE_SAMPLES
* minimal_segment_length_noise
* minimal_segment_length_harmonic
* CONFIDENCE_THRESHOLD

In [ ]:
#setting up
CONFIDENCE_THRESHOLD= 0.9

#get numpy arrays from the tensors
harmonic=harmonic_tensor.numpy()[0]
noise=noise_tensor.numpy()[0]
#get the f0 confidence
confidence_short=audio_features['f0_confidence']

#spread the confidence array to be equal length as the audio
length=len(harmonic)
spreading_factor=length/len(confidence_short)
print("spreading factor: " + str(spreading_factor))
confidence_list=[]
for val in confidence_short:
  confidence_list+=([val]*int(spreading_factor))
confidence=np.array(confidence_list)

##Bad example of hard mixing



In [ ]:
#create a hard mix of the two signals
harmonic_confidence=harmonic*(confidence>CONFIDENCE_THRESHOLD)
noise_confidence=noise*(confidence<CONFIDENCE_THRESHOLD)
mix=harmonic_confidence+noise_confidence

#plots
fig, ax = plt.subplots(nrows=6,
                       ncols=1, 
                       sharex=True,
                       figsize=(30,5))
ax[0].plot(harmonic)
ax[1].plot(noise)
ax[2].plot(confidence)
ax[3].plot(harmonic_confidence)
ax[4].plot(noise_confidence)
ax[5].plot(mix)

play(mix)

##cleaning up the confidence array
Only keeping the changes that are followed by a significantly long array of 1 or 0s

In [ ]:
def clean_confidence(on_off,threshold_noise,threshold_harmonic):
  #args:
  #-on_off: numpy array of zeros and ones that has the same lenght as the audio piece
  #-threshold: int ,mimal number of consecutive ones or zeros such that it remains
  #returns: numpy array where all the short lived changes are cleaned up
  threshold=max(threshold_noise,threshold_harmonic)
  zero_to_one=[]
  one_to_zero=[]
  skip_one=False
  highest_index=len(on_off)
  for i,v in enumerate(on_off):
    if(i<highest_index-threshold):
      if (v==0 and on_off[i+1]==1):
        if skip_one:
          skip_one=False
        else:
          if(check_length(on_off[i+1:i+1+threshold_harmonic],1)):
            zero_to_one.append(i)
          else:
            skip_one=True
      elif (v==1 and on_off[i+1]==0):
        if skip_one:
          skip_one=False
        else:
          if(check_length(on_off[i+1:i+1+threshold_noise],0)):
            one_to_zero.append(i)
          else:
            skip_one=True
  return one_to_zero, zero_to_one


def check_length(on_off_snippet,value):
  for i in on_off_snippet:
    if i!=value:
      return False
  return True

##create a mix of with soft transitions

In [ ]:
#create a soft mix with crossfades of the two
CROSSFADE_SAMPLES=500

watch_window=1000000000 #this is only needed as paramter to change the view of the graphs
#minimal_segment_length=int(np.ceil(CROSSFADE_SAMPLES/2)) #creates too many drops
minimal_segment_length_noise=10000
minimal_segment_length_harmonic=1000


#create an array of 1 and zero and see how long the gaps are
on_off=[1]*(confidence>CONFIDENCE_THRESHOLD)
zero_to_one=[]
one_to_zero=[]
highest_index=len(on_off)
for i,v in enumerate(on_off[:watch_window]):
  if(i<highest_index-CROSSFADE_SAMPLES):
    if (v==0 and on_off[i+1]==1):
      zero_to_one.append(i)
    elif (v==1 and on_off[i+1]==0):
      one_to_zero.append(i)


clean_one_to_zero,clean_zero_to_one=clean_confidence(on_off[:watch_window],minimal_segment_length_noise,minimal_segment_length_harmonic)
#transform it into an array that only saves the index of the first in a change
#plots
fig, ax = plt.subplots(nrows=3,
                       ncols=1, 
                       sharex=True,
                       figsize=(30,10))
ax[0].plot(on_off[:watch_window])
ax[1].vlines(zero_to_one,ymin=0,ymax=1,colors="red")
ax[1].vlines(one_to_zero,ymin=0,ymax=1,colors="green")
ax[2].vlines(clean_zero_to_one,ymin=0,ymax=1,colors="red")
ax[2].vlines(clean_one_to_zero,ymin=0,ymax=1,colors="green")

In [ ]:
import math

#implement equal power solution from https://dsp.stackexchange.com/questions/14754/equal-power-crossfade
def crossFadeEqualVoltage(t):
  volumes=[0,0]
  volumes[0]=math.sqrt(0.5*(1+t))
  volumes[1]=math.sqrt(0.5*(1-t))
  return volumes

def crossFadeLinear(t):
  volumes=[0,0]
  volumes[0]=(t+1)/2
  volumes[1]=(-t+1)/2
  return volumes

#using the S curve (a half of a sine) for the fade
def crossFadeS(t):
  t=t/2+0.5
  volumes=[0,0]
  volumes[0]=np.sin((np.pi * t)-(np.pi/2))/2 + 0.5
  volumes[1]=1-volumes[0]
  return volumes

#using the S curve (a half of a sine) for the fade
#shifting the mixing curves such that the decay starts only when the other signal is fully faded in
def crossFadeS2(t):
  volumes=[0,0]
  if t<0:
    t_norm=t+1
    volumes[0]=1
    volumes[1]=np.sin((np.pi * t)+(np.pi/2))/2 + 0.5
  else:
    volumes[0]=np.sin((np.pi * t)+(np.pi/2))/2 + 0.5
    volumes[1]=1
  return volumes

def fadeIn(audio,CROSSFADE_SAMPLES):
  for t in range(0,CROSSFADE_SAMPLES):
    audio[t]=audio[t]*(np.sin((np.pi*t/CROSSFADE_SAMPLES)-(np.pi/2))/2+0.5)
  return audio

def mix_two(audio1,audio2,cut,cross_length,volumes1,volumes2):
  half_cl=int(cross_length/2)
  crossing=(audio1[cut-half_cl:cut+half_cl]*volumes1)+(audio2[cut-half_cl:cut+half_cl]*volumes2)
  result=np.append(audio1[:cut-half_cl],crossing)
  result=np.append(result,audio2[cut+half_cl:])
  return result

In [ ]:
volumes1=[]
volumes2=[]
for i in range(-int(CROSSFADE_SAMPLES/2),int(CROSSFADE_SAMPLES/2)):
  t=i/(CROSSFADE_SAMPLES/2)
  cr=crossFadeS2(t)
  volumes1.append(cr[0])
  volumes2.append(cr[1])
volumes1=np.array(volumes1)
volumes2=np.array(volumes2)
plt.plot(volumes1)
plt.plot(volumes2)

if on_off[0]==0: #track starts with noise
  mix=noise
  mix=fadeIn(mix,CROSSFADE_SAMPLES)
  for i,v in enumerate(clean_zero_to_one):
    mix=mix_two(mix,harmonic,v,CROSSFADE_SAMPLES,volumes1,volumes2)
    if(i<len(clean_one_to_zero)):
      mix=mix_two(mix,noise,clean_one_to_zero[i],CROSSFADE_SAMPLES,volumes1,volumes2)
else:
  mix=fadeIn(mix,CROSSFADE_SAMPLES)
  for i,v in enumerate(clean_one_to_zero):
    mix=mix_two(mix,noise,v,CROSSFADE_SAMPLES,volumes1,volumes2)
    if(i<len(clean_zero_to_one)):
      mix=mix_two(mix,harmonic,clean_zero_to_one[i],CROSSFADE_SAMPLES,volumes1,volumes2)
      
print("harmonic")
play(harmonic)
print(len(harmonic))
print("noise")
play(noise)
print('Original Resynthesis')
original_resynthesis = harmonic_tensor + noise_tensor
play(original_resynthesis)
print("my mix")
play(mix)
print(len(mix))

 #note: the audible click at the beginning comes from the audio player (is there even if the audio is only 0s)